# Setting up the computational model
To simulate the motion of molecules according to the kinetic theory of gases, we need to set up a dynamic n-body simulation with collisions. Note that the objective of this article is to produce a simulation for pedagogical purposes. Therefore, the code is set up to maximize understanding, not execution speed.

## Defining molecular properties
We start by importing all the required modules.

In [1]:
import os
import sys
import time
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

First, we will define a class called Molecule. Objects of this class will store properties such as the masses, positions and velocities of molecules in the simulation. We also define an attribute called color, that may be used to distinguish between different gases and set the color of the points in the animation.

In [ ]:
class Molecule(object):
    #Constructor
    def __init__(self, molecule, position, velocity, mass, color="black"):
        self.molecule = molecule
        self.position = np.array([x_i for x_i in position])
        self.velocity = np.array([v_i for v_i in velocity])
        self.mass = mass
        self.color = color
    #-------------------------------------------------------------------------------- 
    #Setters for position, velocity, mass and color
    def set_position(self, position):
        self.position = np.array([x_i for x_i in position])
    #-------------------------------------------------------------------------------- 
    def set_velocity(self,velocity):
        self.velocity = np.array([v_i for v_i in velocity])
    #--------------------------------------------------------------------------------   
    def set_color(self, color):
        self.color = color
    #--------------------------------------------------------------------------------
    def set_mass(self, mass):
        self.mass = mass
    #--------------------------------------------------------------------------------   
    #Getters for position, velocity, mass and color of the particle
    def get_position(self):
        return self.position
    #--------------------------------------------------------------------------------
    def get_velocity(self):
        return self.velocity
    #--------------------------------------------------------------------------------    
    def get_color(self):
        return self.color
    #--------------------------------------------------------------------------------    
    def get_mass(self):
        return self.mass

## Adding molecules to a box
Next, we create a Simulation class and define key input parameters, such as the dimensions of the simulation box, that ultimately control the volume of the gas. We also initialize variables for bookkeeping, such as for the wall momentum, that will allow us to calculate pressure.

To add molecules, we first need to initialize their positions and velocities. For both, we need to define functions that create arrays of values based on a distribution input by the user. Note that it does not matter what distributions the initialized positions and velocities follow (as long as there is nothing unphysical, like a molecule outside the box). We shall see that the velocities eventually follow the same distribution.

It is also time to add the molecules to the box. We call the two previously defined functions to generate arrays of initial positions and velocities, create objects of the Molecule class, and assign them to the objects.

Finally, we write a function for general bookkeeping, that is, to make matrices to store the position and velocity vectors as well as their magnitudes. We also make a distance matrix, that stores the distance between every two molecules. This will come in handy to detect collisions.

In [ ]:
class Simulation(object):
    # Constructor
    def __init__(self, name, box_dim, t_step, particle_radius):
        # Set simulation inputs
        self.name = name # Name of the simulation
        self.box_dim = [x for x in box_dim] # Dimensions of the box
        self.t_step = t_step # Timestep
        self.particle_radius = particle_radius # Radius of the particles
        
        # Calculate volume and number of dimensions
        self.V = np.prod(self.box_dim) # Area/Volume of the box
        self.dim = int(len(box_dim)) # Number of dimensions (2D or 3D)
        
        # Initialize paramters
        self.molecules = [] # Create empty list to store objects of class Molecule
        self.n_molecules = 0 # Create variable to store number of molecules
        self.wall_collisions = 0 # Create variable to store number of wall collisions
        self.wall_momentum = 0 # Create variable to store net momentum exchanged with wall
    #--------------------------------------------------------------------------------
    def _generate_initial_positions(self, n, dist="uniform"):
            #Uniform distribution
            if dist == "uniform":
                _pos = np.random.uniform(low=[0]*self.dim, high=self.box_dim, size=(n,self.dim))
                
            #Store positions in temporary variable    
            self._positions = _pos
    #--------------------------------------------------------------------------------
    def _generate_initial_velocities(self, n, v_mean, v_std, dist="normal"):
        #Normal distribution with mean v_mean and std v_std
        if dist == "normal":
            self.v_mean = v_mean
            self.v_std = v_std
            _vel=np.random.normal(loc=v_mean, scale=v_std, size=(n,self.dim))
        
        #Uniform distribution with lower bound v_mean and higher bound v_std
        if dist == "uniform":
            self.v_mean = v_mean
            self.v_std = v_std
            _vel = np.random.uniform(low=v_mean, high=v_std, size=(n,self.dim))
        
        #All velocities equal to v_mean
        if dist == "equal":
            self.v_mean = v_mean
            self.v_std = v_std
            _vel = v_mean*np.ones((n,self.dim))
        
        #Randomly switch velocities to negative with probability 0.5
        for i in range(_vel.shape[0]):
            for j in range(_vel.shape[1]):
                if np.random.uniform() > 0.5:
                    _vel[i,j] = -_vel[i,j]
        
        #Store velocities in temporary variable
        self._velocities = _vel
    #--------------------------------------------------------------------------------
    def add_molecules(self, molecule, n, v_mean, v_std, pos_dist="uniform", v_dist="normal", color="black"):
        #Generate initial positions and velocities
        self._generate_initial_positions(n, dist = pos_dist)
        self._generate_initial_velocities(n,v_mean,v_std, dist = v_dist)
        
        #Initialize objects of class Molecule in a list (set mass to 1 as default)
        _add_list=[Molecule(molecule, position = self._positions[i,:], velocity = self._velocities[i,:], \
            color = color, mass = 1) for i in range(n)]
        self.molecules.extend(_add_list)
        self.n_molecules += len(_add_list)
    #--------------------------------------------------------------------------------
    def make_matrices(self):
        # Make empty matrices to store positions, velocities, colors, and masses
        self.positions=np.zeros((self.n_molecules, self.dim))
        self.velocities=np.zeros((self.n_molecules, self.dim))
        self.colors=np.zeros(self.n_molecules, dtype="object")
        self.masses=np.zeros(self.n_molecules)
        
        #Iterate over molecules, get their properties and assign to matrices
        for i,m in enumerate(self.molecules):
            self.positions[i,:] = m.get_position()
            self.velocities[i,:] = m.get_velocity()
            self.colors[i] = m.get_color()
            self.masses[i] = m.get_mass()
        
        #Make vectors with magnitudes of positions and velocities
        self.positions_norm = np.linalg.norm(self.positions, axis=1)
        self.velocities_norm = np.linalg.norm(self.velocities, axis=1)
        
        #Make distance matrix
        self.distance_matrix = np.zeros((self.n_molecules, self.n_molecules))
        for i in range(self.distance_matrix.shape[0]):
            for j in range(self.distance_matrix.shape[1]):
                self.distance_matrix[i,j] = np.linalg.norm(self.positions[i,:]-self.positions[j,:])
                
        #Set diagonal entries (distance with itself) to a high value
        #to prevent them for appearing in the subsequent distance filter
        np.fill_diagonal(self.distance_matrix, 1e5)
    #--------------------------------------------------------------------------------
    def update_positions(self):
        #### 1: Check molecule collisions
        # Find molecule pairs that will collide
        collision_pairs = np.argwhere(self.distance_matrix < 2 * self.particle_radius)
        
        # If collision pairs exist 
        if len(collision_pairs):
            # Go through pairs and remove repeats of indices
            # (for eg., only consider (1,2), remove (2,1))
            pair_list = []
            for pair in collision_pairs:
                add_pair = True
                for p in pair_list:
                    if set(p) == set(pair):
                        add_pair = False
                        break
                if add_pair:
                    pair_list.append(pair)

            #For every remaining pair, get the molecules, positions, and velocities
            for pair in pair_list:
                m_1 = self.molecules[pair[0]] # Get molecules ("m")
                m_2 = self.molecules[pair[1]]
                
                x_1 = m_1.get_position() # Get positions ("x")
                x_2 = m_2.get_position()
                
                u_1 = m_1.get_velocity() # Get velocities ("u")
                u_2 = m_2.get_velocity()
                
                # Check if molecules are approaching or departing
                approach_sign = np.sign(np.dot(u_1-u_2,x_2-x_1))
                
                # If molecules are approaching
                if approach_sign == 1:
                    # Get masses ("ms" to differentiate between mass "ms" and molecule "m")
                    ms_1 = m_1.get_mass()
                    ms_2 = m_2.get_mass()
                    
                    #Calculate final velocities
                    v_1 = u_1 - 2*ms_2/(ms_1 + ms_2) * (np.dot(u_1 - u_2, x_1 - x_2)\
                        /np.linalg.norm(x_1 - x_2)**2) * (x_1 - x_2)
                    v_2 = u_2 - 2*ms_1/(ms_1 + ms_2) * (np.dot(u_2 - u_1, x_2 - x_1)\
                        /np.linalg.norm(x_2 - x_1)**2) * (x_2 - x_1)
                    
                    #Update velocities of the molecule objects
                    m_1.set_velocity(v_1)
                    m_2.set_velocity(v_2)
        
        #### 2: Update positions
        # I terate over all the molecule objects
        for i,m in enumerate(self.molecules):
            # Get the position, velocity, and mass
            _x = m.get_position()
            _v = m.get_velocity()
            _m = m.get_mass()
            
            # Calculate new position
            _x_new = _x + _v * self.t_step
            
            #### 3: Check wall collisions
            # Check collisions with the top and right walls
            _wall_diff = _x_new - np.array(self.box_dim)           
            # If wall collisions present
            if _wall_diff[_wall_diff>=0].shape[0] > 0:
                # Increment collision counter
                self.wall_collisions += 1
                # Check whether collision in x or y direction
                _coll_ind = np.argwhere(_wall_diff>0)
                # For component(s) to be reflected
                for c in _coll_ind:
                    # Reflect velocity
                    _v[c]=-_v[c]
                    # Increment wall momentum
                    self.wall_momentum += 2 * _m * np.abs(_v[c])
                # Update velocity
                m.set_velocity(_v)
                # Update position based on new velocity
                _x_new = _x + _v * self.t_step
            
            # Check collisions with the bottom and left walls    
            if _x_new[_x_new <= 0].shape[0] > 0:
                # Increment collision counter
                self.wall_collisions += 1
                # Check whether collision in x or y direction
                _coll_ind = np.argwhere(_x_new < 0)
                # or component(s) to be reflected
                for c in _coll_ind:
                    # Reflect velocity
                    _v[c] = -_v[c]
                    # Increment wall momentum
                    self.wall_momentum += 2 * _m * np.abs(_v[c])
                # Update velocity    
                m.set_velocity(_v)
                # Update position based on new velocity
                _x_new = _x + _v * self.t_step
                
            # Update position of the molecule object            
            m.set_position(_x_new)
        
        # Construct matrices with updated positions and velocities
        self.make_matrices()
    #--------------------------------------------------------------------------------
    def safe_division(self, n, d):
        if d == 0:
            return 0
        else:
            return n/d
    #--------------------------------------------------------------------------------
    def run_simulation(self, max_time):
        # Print "Starting simulation"
        print("Starting simulation...")

        # Make matrices
        self.make_matrices()
        
        # Calculate number of iterations
        self.max_time = max_time
        self.n_iters = int(np.floor(self.max_time/self.t_step))
        
        # Make tensors to store positions and velocities of all molecules at each timestep f
        self.x_dynamics = np.zeros(((self.n_molecules, self.dim, self.n_iters)))
        self.v_dynamics = np.zeros((self.n_molecules, self.n_iters))
        
        #In each iteration
        for i in range(self.n_iters):                   
            #Save positions and velocities to the defined tensors
            self.x_dynamics[:,:,i] = self.positions
            self.v_dynamics[:,i] = self.velocities_norm
            
            #Calculate rms velocity
            self.v_rms = np.sum(np.sqrt(self.velocities_norm ** 2))/self.velocities_norm.shape[0]
            
            #Print current iteration information
            _P = self.safe_division(self.wall_momentum, i * self.t_step * np.sum(self.box_dim))
            print("Iteration:{0:d}\tTime:{1:.2E}\tV_RMS:{2:.2E}\tWall Pressure:{3}".format(i,i*self.t_step,self.v_rms,_P))
           
            # Call the update_positions function to handle collisions and update positions
            self.update_positions()
            
        # Caclulate final pressure
        self.P = self.wall_momentum/(self.n_iters * self.t_step * np.sum(self.box_dim))
        print("Average pressure on wall: {0}".format(self.P))
        return self.P
    #--------------------------------------------------------------------------------
    #Still in Simulation class
    def create_2D_box(self):
        fig=plt.figure(figsize=(10,10*self.box_dim[1]/self.box_dim[0]),dpi=300)
        return fig

## Detecting and handling collisions
Here, we come to the meat of the simulation, that is, modeling the dynamics of the molecules. Collision physics are defined for a 2D plane, but extension to a 3D box is possible with minor changes. We define a function that goes through a three-item checklist:
- Check if there are molecule-molecule collisions and if yes, update velocities accordingly
- Update positions of the molecules based on their velocities
- Check if there are molecule-wall collisions and if yes, update velocities and wall momentum accordingly

To filter molecules that are close enough to collide, we scour the distance matrix and return the pairs of indices of molecules that have distances less than the sum of their radii. It is possible for molecules to be within this cutoff, and yet be departing away from each other. So, we apply another criterion (see Figure 1) to investigate whether molecules are approaching each other. If they are, we update their velocities based on the equations given in Figure 1. We arrive at these equations by conserving the kinetic energies and the linear momenta (along the collision axis) of the molecules.

![Figure 1](assets/Figure%201.webp)

Figure 1: Schematic showing collision of two spheres in a 2D plane. Variables and equations in blue represent tangential and normal components of velocities (to the collision axis). Variables in bold are vectors, the rest are scalars. Angular brackets indicate dot (inner) product.

Updating the positions of the molecules is quite simple. We add the product of the velocity vector and the timestep to the previous position to get the new position. To identify collisions with the walls, we simply check if the new position of a molecule exceeds either the lower or upper bounds of the dimensions of the box. If it does, we change the sign of the velocity normal to the wall, and set the new position based on this velocity. Further, we add twice the magnitude of this velocity to the variable tracking the momentum exchanged with the wall (See the update_positions function in the Simulation class).

That’s it, the hard part is done! Finally, we wrote a function to run the simulation (See the safe_division and the run_simulation functions in the Simulation class). This involves calling the functions that we have defined previously in a loop that runs for a specified number of iterations, calculated based on the specified simulation time and time step.

That concludes the code involving the physics of the simulation. Running a simulation is no fun, however, if you cannot visualize it. Let’s utilize matplotlib to create an animation of the simulation.

## Animating the simulation box
We start by writing a function to create a figure with an aspect ratio that is consistent with the provided dimensions of the box.